In [456]:
#Description: Build a book recommendation engine/system

In [457]:
#Import the libraries
import numpy as np
import pandas as pd
import random
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [458]:
#Store the data
# sequence=[1,2,3,4,5]
# random.choice(sequence)
# choice = random.choice(sequence)
# df = pd.read_csv('books' + str(choice) + '.csv', encoding='unicode_escape', error_bad_lines=False, delimiter=";")
df = pd.read_csv('books1.csv', encoding='unicode_escape', error_bad_lines=False, delimiter=";")

C:\Users\jadkh\AppData\Local\Temp\ipykernel_21884\1641578132.py:6: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('books1.csv', encoding='unicode_escape', error_bad_lines=False, delimiter=";")
b'Skipping line 6452: expected 8 fields, saw 9\n'


In [459]:
#Show the data
df

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
9993,0374172897,The Hours : A Novel,Michael Cunningham,1998,"Farrar, Straus and Giroux",http://images.amazon.com/images/P/0374172897.0...,http://images.amazon.com/images/P/0374172897.0...,http://images.amazon.com/images/P/0374172897.0...
9994,0140283404,Beloved (Penguin Great Books of the 20th Century),Toni Morrison,2000,Penguin Books,http://images.amazon.com/images/P/0140283404.0...,http://images.amazon.com/images/P/0140283404.0...,http://images.amazon.com/images/P/0140283404.0...
9995,0380730774,Read This and Tell Me What It Says : Stories (...,A. Manette Ansay,1998,William Morrow,http://images.amazon.com/images/P/0380730774.0...,http://images.amazon.com/images/P/0380730774.0...,http://images.amazon.com/images/P/0380730774.0...
9996,0862418879,The Star Rover,Jack London,2000,Canongate Books,http://images.amazon.com/images/P/0862418879.0...,http://images.amazon.com/images/P/0862418879.0...,http://images.amazon.com/images/P/0862418879.0...


In [460]:
#Create a list of columns to keep
columns = ['title', 'author', 'publisher']

In [461]:
#Create a function to combine the important columns/features
def combine_features(data):
  features = []
  for i in range(0, data.shape[0]):
    features.append(str(data['Book-Title'][i]) + ' ' + str(data['Book-Author'][i]) + ' ' + str(data['Publisher'][i]))

  return features

In [462]:
#Create a column to store the combined features
df['combined_features'] = combine_features(df)
#Show the data
df

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,combined_features
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,Classical Mythology Mark P. O. Morford Oxford ...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,Clara Callan Richard Bruce Wright HarperFlamin...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,Decision in Normandy Carlo D'Este HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,Flu: The Story of the Great Influenza Pandemic...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,The Mummies of Urumchi E. J. W. Barber W. W. N...
...,...,...,...,...,...,...,...,...,...
9993,0374172897,The Hours : A Novel,Michael Cunningham,1998,"Farrar, Straus and Giroux",http://images.amazon.com/images/P/0374172897.0...,http://images.amazon.com/images/P/0374172897.0...,http://images.amazon.com/images/P/0374172897.0...,"The Hours : A Novel Michael Cunningham Farrar,..."
9994,0140283404,Beloved (Penguin Great Books of the 20th Century),Toni Morrison,2000,Penguin Books,http://images.amazon.com/images/P/0140283404.0...,http://images.amazon.com/images/P/0140283404.0...,http://images.amazon.com/images/P/0140283404.0...,Beloved (Penguin Great Books of the 20th Centu...
9995,0380730774,Read This and Tell Me What It Says : Stories (...,A. Manette Ansay,1998,William Morrow,http://images.amazon.com/images/P/0380730774.0...,http://images.amazon.com/images/P/0380730774.0...,http://images.amazon.com/images/P/0380730774.0...,Read This and Tell Me What It Says : Stories (...
9996,0862418879,The Star Rover,Jack London,2000,Canongate Books,http://images.amazon.com/images/P/0862418879.0...,http://images.amazon.com/images/P/0862418879.0...,http://images.amazon.com/images/P/0862418879.0...,The Star Rover Jack London Canongate Books


In [463]:
#Create a function to add ids so we can enumerate later
def ids(data):
  features = []
  for i in range(0, data.shape[0]):
    features.append(i)

  return features

In [464]:
#Create a column to store the ids
df['book_id'] = ids(df)
#Show the data
df

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,combined_features,book_id
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,Classical Mythology Mark P. O. Morford Oxford ...,0
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,Clara Callan Richard Bruce Wright HarperFlamin...,1
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,Decision in Normandy Carlo D'Este HarperPerennial,2
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,Flu: The Story of the Great Influenza Pandemic...,3
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,The Mummies of Urumchi E. J. W. Barber W. W. N...,4
...,...,...,...,...,...,...,...,...,...,...
9993,0374172897,The Hours : A Novel,Michael Cunningham,1998,"Farrar, Straus and Giroux",http://images.amazon.com/images/P/0374172897.0...,http://images.amazon.com/images/P/0374172897.0...,http://images.amazon.com/images/P/0374172897.0...,"The Hours : A Novel Michael Cunningham Farrar,...",9993
9994,0140283404,Beloved (Penguin Great Books of the 20th Century),Toni Morrison,2000,Penguin Books,http://images.amazon.com/images/P/0140283404.0...,http://images.amazon.com/images/P/0140283404.0...,http://images.amazon.com/images/P/0140283404.0...,Beloved (Penguin Great Books of the 20th Centu...,9994
9995,0380730774,Read This and Tell Me What It Says : Stories (...,A. Manette Ansay,1998,William Morrow,http://images.amazon.com/images/P/0380730774.0...,http://images.amazon.com/images/P/0380730774.0...,http://images.amazon.com/images/P/0380730774.0...,Read This and Tell Me What It Says : Stories (...,9995
9996,0862418879,The Star Rover,Jack London,2000,Canongate Books,http://images.amazon.com/images/P/0862418879.0...,http://images.amazon.com/images/P/0862418879.0...,http://images.amazon.com/images/P/0862418879.0...,The Star Rover Jack London Canongate Books,9996


In [465]:
#Convert the text from the new column to a matrix of word counts
cm = CountVectorizer().fit_transform(df['combined_features'])

In [466]:
#Get the cosine similary matrix from the count matrix
cs = cosine_similarity(cm)
#Print the scores
print(cs)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [467]:
#Get the id of the book the reader likes create a list of tuples in the form (id, similarity score)
def getTupleList(book_isbn):
    try: 
        book_id = df[df['ISBN'] == book_isbn]['book_id'].values[0]
        scores = list(enumerate(cs[book_id]))
        return scores
    except: 
        return "book not found"
    

In [468]:
#Sort the list of similar books in description order
def sortTupleList(tuple_list):
    if(tuple_list == "book not found"):
        return "book not found"
    else:
        sorted_scores = sorted(tuple_list, key = lambda x:x[1], reverse = True)
        sorted_scores = sorted_scores[1:]
        return sorted_scores

In [469]:
#Created a loop to print the first 5 books from the sorted list
def getSuggestions(tuple_list):
  if(tuple_list == "book not found"):
    return "book not found"
  else: 
    list = {}
    for item in tuple_list:
      book_title = df[df.book_id == item[0]]['Book-Title'].values[0] 
      book_isbn = df[df.book_id == item[0]]['ISBN'].values[0]             
      list[str(book_isbn)] = {}
      list[str(book_isbn)]['Book-Title'] = book_title
      if len(list) == 15:
        return list

In [470]:
from flask import Flask
app = Flask(__name__)

@app.route('/<isbn>')
def get(isbn):
    return getSuggestions(sortTupleList(getTupleList(isbn)))

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000 (Press CTRL+C to quit)
127.0.0.1 - - [09/May/2022 11:35:43] "GET /184668286X HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2022 11:35:43] "GET /0304357960 HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2022 11:35:43] "GET /198211794X HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2022 11:35:43] "GET /1885477228 HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2022 11:35:43] "GET /0007108222 HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2022 11:35:43] "GET /2012026656 HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2022 11:35:43] "GET /0836827716 HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2022 11:35:43] "GET /1101141743 HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2022 11:35:43] "GET /0375708820 HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2022 11:35:43] "GET /1607747510 HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2022 11:35:43] "GET /0064431436 HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2022 11:35:43] "GET /0915145774 HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2022 11:35:43] "GET /0385530218 HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2022 11:3